###Project Summary
<img src ='https://images.unsplash.com/photo-1548859047-1d15def63a14?ixlib=rb-1.2.1&ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&auto=format&fit=crop&w=2700&q=80' width="800" height = "200">
######Ivy(Ruxin) Tong | November 15th, 2020 | Image courtesy of Ivan Olenkevich"

####**Objective** : Address the Challenge of Evaluation and Prediction of American’s Mobility under Extreme Weather Events






####**Datasets** :  Weather_CBG_2019 | Social Distancing Metrics v2.1 
######***About the data***
- Dataset - Weather_CBG_2019 records daily weather parameters by census block group in the United States. There're 7 variables. 

| Variable   | Description  | Unit | 
|-----------------  |---------------|------|
| geoid    | census block group ID |  | 
| precip  | daily precipitation | mm|
| rmax | maximum daily relative humidity | % |
| rmin | mminimum daily relative humidity | % |
| srad | surface downwelling solar radiation | W/m^2 |
| tmax | maximum daily temperature | degress F |
| tmin | minimum daily temperature | degress F |
| wind_speed | wind speed | mph|


- Dataset - Social Distancing Metrics v2.1 is a product of Safegraph which aggregately summarizes daily views of USA foot-traffic between census block groups. There're 23 variables. For this analysis, I mainly use
  
  
| Variable   | Description  | Unit | 
|-----------------  |---------------|------|
| origin_census_block_group    | 12-digit FIPS code for the Census Block Group |  | 
| distance_traveled_from_home  | Median distance traveled from the geohash-7 of the home by the devices measured within the time period. All distance with 0 has been excluded. | m |


  
  
######***Acknowledgements*** : Social Distancing Metrics v2.1 is downloaded from Safegraph



####**Conclusion**:




####**Model and Result**:

#### Analysis
###### Step 0 : Install and Import Relevant Packages

In [4]:
%run ./Packages_setup


###### Step 1 : Data Preparation and Analysis for weather_cbg_2019
  - connect AWS S3 Bucket to Databricks and read the dataset

In [6]:
# Secure access to S3 buckets using instance profiles
awsAccessKey = ""
# Encode the Secret Key to remove any "/" characters
secretKey = "".replace("/", "%2F")
awsBucketName = ""
mountPoint = f"/mnt/weathercbg"


mountTarget = "s3a://{}:{}@{}".format(awsAccessKey, secretKey, awsBucketName)
dbutils.fs.mount(mountTarget, mountPoint)


Out[4]: True

In [7]:
%fs ls /mnt/weathercbg

path,name,size
dbfs:/mnt/weathercbg/cbg.geojson,cbg.geojson,3087341904
dbfs:/mnt/weathercbg/weather_cbg_2019.csv,weather_cbg_2019.csv,6071633167


In [8]:
dbutils.fs.head("dbfs:/mnt/weathercbg/weather_cbg_2019.csv",1000)

[Truncated to first 1000 bytes]
Out[5]: 'geoid,date,precip,rmax,rmin,srad,tmin,tmax,wind_speed\r\n10730059033,2019-01-01,0,97.2,65.2,111.8,48.47,61.97,2.46062992125984\r\n10730059033,2019-01-02,20.9,86.3,81.6,30.6,48.47,52.79,3.3554044380816\r\n10730059033,2019-01-03,24.9,79.4,63.1,59.2,50.63,59.4499999999999,5.81603435934145\r\n10730059033,2019-01-04,0.9,100,55.7,99.1,40.5499999999999,62.1499999999999,14.3163922691482\r\n10730059033,2019-01-05,0,87.6,39.2,126.8,36.9499999999999,60.17,7.38188976377953\r\n10730059033,2019-01-06,0,95.8,33.2,119.4,36.77,66.83,2.90801717967072\r\n10730059033,2019-01-07,0.4,83.8,38.3,126.5,44.1499999999999,67.37,7.15819613457409\r\n10730059033,2019-01-08,0,100,43.9,116.4,42.71,68.99,8.94774516821761\r\n10730059033,2019-01-09,0,68.6,34,135,31.73,51.53,10.9609878310666\r\n10730059033,2019-01-10,0,60.1,31.1,133.5,27.41,45.5899999999999,8.05297065139585\r\n10730059033,2019-01-11,0,73.2,28.3,131.5,27.41,52.97,7.38188976377953\r\n10730059033,2019-01-12,17.6,60.1,54.5,52.8,45.77,50.81,9.84251968503937\r\n10730059033,2019-01-13,0,'

In [9]:
# read weathercbgDF

weathercbgDF = (spark.read
  .option("delimiter", ",")
  .option("header", True)
  .option("inferSchema", True)
  .csv("/mnt/weathercbg/weather_cbg_2019.csv")
)


- Transform dataset

In [11]:
# Add three date-related variables
weathercbgDF = weathercbgDF.withColumn("month", F.month('date'))
weathercbgDF = weathercbgDF.withColumn("day", F.dayofmonth('date'))
weathercbgDF = weathercbgDF.withColumn("year", F.year('date'))
weathercbgDF.show(5)

+-----------+----------+------+-----+----+-----+----------------+----------------+----------------+-----+---+----+
 geoid| date|precip| rmax|rmin| srad| tmin| tmax| wind_speed|month|day|year|
+-----------+----------+------+-----+----+-----+----------------+----------------+----------------+-----+---+----+
10730059033|2019-01-01| 0.0| 97.2|65.2|111.8| 48.47| 61.97|2.46062992125984| 1| 1|2019|
10730059033|2019-01-02| 20.9| 86.3|81.6| 30.6| 48.47| 52.79| 3.3554044380816| 1| 2|2019|
10730059033|2019-01-03| 24.9| 79.4|63.1| 59.2| 50.63|59.4499999999999|5.81603435934145| 1| 3|2019|
10730059033|2019-01-04| 0.9|100.0|55.7| 99.1|40.5499999999999|62.1499999999999|14.3163922691482| 1| 4|2019|
10730059033|2019-01-05| 0.0| 87.6|39.2|126.8|36.9499999999999| 60.17|7.38188976377953| 1| 5|2019|
+-----------+----------+------+-----+----+-----+----------------+----------------+----------------+-----+---+----+
only showing top 5 rows

In [12]:
def spark_df_shape(self):
    return (self.count(),len(self.columns))
  
pyspark.sql.dataframe.DataFrame.shape = spark_df_shape

print("weather cbg 2019 dataset")
weathercbgDF.printSchema()
print("shape:")
print(weathercbgDF.shape())

from pyspark.sql.functions import min, max

display(
  weathercbgDF.select(min("date"), max("date"))
)



min(date),max(date)
2019-01-01,2019-12-31


In [13]:
#  create monthly table, for visualization and faster operation

weather_cbg_month = (weathercbgDF.groupBy("geoid","month")
                            .agg(
                              mean("precip").alias("mean_precip"),
                              mean("rmax").alias("mean_rmax"),
                              mean("rmin").alias("mean_rmin"),
                              mean("srad").alias("mean_srad"),
                              mean("tmin").alias("mean_tmin"),
                              mean("tmax").alias("mean_tmax"),
                              mean("wind_speed").alias("mean_wind_speed"))
                             .sort(col("geoid").asc(),col("month").desc()))

weather_cbg_month.show(5)

+-----------+-----+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+
 geoid|month| mean_precip| mean_rmax| mean_rmin| mean_srad| mean_tmin| mean_tmax| mean_wind_speed|
+-----------+-----+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+
10010201001| 12|5.141935483870967|90.35806451612902|49.258064516129025|102.40967741935484|41.194516129032216|61.98161290322579| 9.532234973560858|
10010201001| 11|3.246666666666667|94.27666666666667| 41.46666666666666| 140.0166666666667| 36.60799999999996|64.20799999999997| 7.642865664519213|
10010201001| 10| 3.44516129032258|88.64838709677419| 44.99677419354839|175.74516129032253| 57.59193548387094|80.24870967741931| 7.143764287528577|
10010201001| 9| 0.0|92.82666666666667| 33.31|257.41333333333336| 66.242| 94.4| 6.203769983297547|
10010201001| 8|4.241935483870967|99.22258064516129|43.961290322580645| 264.0387096774194| 69.70419354838707|92.40161290322578|5.1160897776341026|
+-----------+-----+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+
only showing top 5 rows

In [14]:
# Generate a sample for exploratory data analysis
weather_cbg_2019_month_sample = weather_cbg_month.sample(0.01).toPandas()
weather_cbg_2019_month_sample.head()


,geoid,month,mean_precip,mean_rmax,mean_rmin,mean_srad,mean_tmin,mean_tmax,mean_wind_speed
0,10010204001,7,2.652,97.506,46.442,309.129,70.047,90.875,6.242
1,10010204002,3,2.665,78.765,39.519,212.565,43.935,67.730,9.078
2,10010204004,7,2.748,97.713,45.610,309.100,70.233,91.502,6.213
3,10010206001,2,4.018,79.996,48.771,136.057,47.801,69.337,9.435
4,10010207001,9,0.000,92.523,33.323,256.807,66.686,94.712,6.241


In [15]:
# Understand distribution of Variables
variables = weather_cbg_2019_month_sample.drop(["geoid","month"], axis = 1)
fig,ax = plt.subplots(1,7, figsize= (30,5))

for i, variable in enumerate(variables):
  sns.distplot(variables[variable], ax=ax[i])
  ax[i].set_xlim(0,250)

In [16]:
# these distributions are not on the same scale, standardize them
fix, ax = plt.subplots(1,7,figsize = (25,5))

for i, variable in enumerate(variables):
  original_variables = variables[variable]
  variable_scaled = (original_variables - original_variables.mean())/original_variables.std()
  sns.distplot(variable_scaled, ax=ax[i])
  ax[i].set_xlim(-2,2)

In [17]:
#correlation
sns.pairplot(variables)


In [18]:
corr = variables.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask,0)] = True
sns.heatmap(corr, mask = mask,square = True, annot = True)

In [19]:
# scaled data by month
for i, variable in enumerate(variables):
  original_variables = variables[variable]
  variable_scaled = (original_variables - original_variables.mean())/original_variables.std()
  sns.lineplot(x = "month", y=variable_scaled, data = weather_cbg_2019_month_sample,legend='brief', label=variable)

plt.ylabel("values")
plt.legend(bbox_to_anchor=(1.02, 1),borderaxespad=0.)
plt.title("mean weather parameters level by month")

  

###### Step 2 : Data Preparation and Analysis for Social Distancing Metrics v2.1
  -